# PV field albedo from percentages

In [1]:
import pandas as pd
import numpy as np
import glob

from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format

In [2]:
# Data location
project_path = './'
data_path = project_path + '../data/'

# Input path
percentage_fn = data_path + 'temperature_by_element.csv'
ketura_fn     = data_path + 'Ketura_all_corr.csv'
yatir_fn      = project_path + '../../data/towerSAS/Yatir_2000-2020.csv'

# Output path
graphs_path = project_path + '../graphs/'

# Constants
albedo_pv = 0.05
albedo_desert = 0.40

In [3]:
def load_tower(fn, silent=False):
    if (not silent):
        print('  -', fn.split('/')[-1])
    temp = pd.read_csv(fn, index_col=None)
    temp.rename({'date_mid_hour': 'DateTime'}, axis=1, inplace=True)
    temp['DateTime'] = pd.to_datetime(temp['DateTime'], format='%d%b%y:%H:%M')
    # Remove obsolete columns
    temp.drop(['year','date','DOY','month','weekNo','mid_hour','mmyy','Bat_V','Hum_AC'], axis=1, inplace=True)
    if (not silent): print("    ", '100.0 %\t', fn.split('/')[-1])
    return(temp)

def load_truck(fn, index_col=False, silent=False):
    if (not silent):
        print('  -', fn.split('/')[-1])
    temp = pd.read_csv(fn, index_col=index_col)
    temp.rename(columns={'date_time': 'DateTime'}, inplace=True)
    temp['DateTime'] = pd.to_datetime(temp['DateTime'], format='%Y-%m-%d %H:%M:%S')
    temp['DateTime'] = temp['DateTime'] + pd.Timedelta(minutes=15)
    return(temp)

def load_percentage(fn, silent=False):
    if (not silent): print('  -', fn.split('/')[-1])
    temp = pd.read_csv(fn)
    
    temp.rename(columns={'date_time': 'DateTime'}, inplace=True)
    temp['DateTime'] = pd.to_datetime(temp['DateTime'], format='%Y-%m-%d %H:%M:%S')
    #temp['DateTime'] = temp['DateTime'] + pd.Timedelta(minutes=15)
    # Move timestamp to front
    col = temp.pop('DateTime')
    temp.insert(0, col.name, col, allow_duplicates=True)
    
    # Remove obsolete columns
    temp.drop(['Unnamed: 0','flight'], axis=1, inplace=True)
    # Change to wide format
    wide = pd.pivot(temp, index='DateTime', columns='label', values='coverage_percent')
    wide.reset_index(inplace=True)
    
    # Normalise the % to 100%, because sometimes it only reaches 96% total
    wide.loc[wide['sun'].isna(), 'sun'] = 0
    wide['total'] = wide['panel'] + wide['shadow'] + wide['soil'] + wide['sun']
    wide['sun']    = wide['sun'] * 100 / wide['total']
    wide['panel']  = wide['panel'] * 100 / wide['total']
    wide['shadow'] = wide['shadow'] * 100 / wide['total']
    wide['soil']   = wide['soil'] * 100 / wide['total']
    wide.drop(['total'], axis=1, inplace=True)
    
    return(wide)

def add_ecosystem(df):
    df = df.copy()
    # Add ecosystem
    df['Ecosystem'] = np.nan
    df.loc[(df['DateTime'] >= '2018-10-16') & (df['DateTime'] < '2018-10-24'), 'Ecosystem'] = 'Desert background'
    df.loc[(df['DateTime'] >= '2018-10-24') & (df['DateTime'] < '2018-11-01'), 'Ecosystem'] = 'PV field'
    
    df.loc[(df['DateTime'] >= '2019-07-09') & (df['DateTime'] < '2019-07-16'), 'Ecosystem'] = 'Desert background'
    df.loc[(df['DateTime'] >= '2019-07-16') & (df['DateTime'] < '2019-07-24'), 'Ecosystem'] = 'PV field'
    # Season
    df['Season'] = np.nan
    df.loc[df['DateTime'].dt.year == 2018, 'Season'] = 'Autumn'
    df.loc[df['DateTime'].dt.year == 2019, 'Season'] = 'Summer'
    # Create day identifier
    df['time'] = df['DateTime'].dt.strftime('%H:%M')
    # shift column 'timestamp' to first position
    col = df.pop('time')
    df.insert(0, col.name, col, allow_duplicates=True)
    col = df.pop('Ecosystem')
    df.insert(0, col.name, col, allow_duplicates=True)
    col = df.pop('Season')
    df.insert(0, col.name, col, allow_duplicates=True)
    col = df.pop('DateTime')
    df.insert(0, col.name, col, allow_duplicates=True)
    return(df)

In [4]:
print('Loading data...')

ketura_df = load_truck(ketura_fn)
ketura_df = add_ecosystem(ketura_df) # TODO

perc_df = load_percentage(percentage_fn)
perc_df = add_ecosystem(perc_df) # TODO

yatir_full_df = load_tower(yatir_fn) # Load 20 years
yatir_df = yatir_full_df.loc[(yatir_full_df['DateTime'].dt.year == 2018) | (yatir_full_df['DateTime'].dt.year == 2019)].copy()
del [yatir_full_df] # Clean up memory

print('Done...')

Loading data...
  - Ketura_all_corr.csv
  - temperature_by_element.csv
  - Yatir_2000-2020.csv


C:\Users\Jonathan\.conda\envs\dp\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.


     100.0 %	 Yatir_2000-2020.csv
Done...


In [5]:
# Collapse percentages to diurnals
perc_df['DateTime'] = pd.to_datetime(perc_df['time'], format='%H:%M')

# set the DateTime column as the index
perc_df.set_index('DateTime', inplace=True)
# resample the data to 15-minute intervals and apply a smoothing function (e.g., rolling mean)
df_resampled = perc_df.groupby(['Ecosystem', 'Season']).resample('15T').mean().rolling(window=6, min_periods=3).mean()
df_resampled.reset_index(inplace=True)

df_resampled['time'] = df_resampled['DateTime'].dt.strftime('%H:%M')
col = df_resampled.pop('time')
df_resampled.insert(0, col.name, col, allow_duplicates=True)
df_resampled.drop(['DateTime'], axis=1, inplace=True)

perc_df.reset_index(inplace=True)
#display(df_resampled)

In [6]:
display(df_resampled.loc[df_resampled['time'] == '12:30'])

label,time,Ecosystem,Season,panel,shadow,soil,sun
24,12:30,PV field,Autumn,NaN,NaN,NaN,NaN
64,12:30,PV field,Summer,49.547527,5.513662,44.938811,0.0


In [7]:
# Calculate diffuse fraction in Yatir (Assuming it's the same in Ketura)
yatir_df['f_dif'] = yatir_df['S_top_diff(CM21_V)_Wm-2'] / yatir_df['S_top_atm(CM21_IV)_Wm-2']

# Add diffuse fraction from Yatir to Ketura
ketura = ketura_df.merge(yatir_df[['DateTime','f_dif']], how='left', on=['DateTime'])

# Add percentages
ketura = ketura.merge(df_resampled, how='left',  on=['Ecosystem','Season','time'])

# Fix bad SW values
ketura.loc[ketura['SW_IN_merge'] < 0, 'SW_IN_merge'] = 0
ketura.loc[ketura['SW_OUT_merge'] < 0, 'SW_OUT_merge'] = 0
ketura.loc[ketura['SW_IN_merge'] <= 0, 'SW_OUT_merge'] = 0

# Re-calculate albedo from SW measurements (relevant for the desert)
ketura['albedo'] = np.nan
ketura['albedo'] = ketura['SW_OUT_merge'] / ketura['SW_IN_merge']

# Extract desert albedo & rename. This will be used for the soil & shade fractions of the PV field
desert_albedo = ketura.loc[ketura['Ecosystem'] == 'Desert background',['time','Season','albedo']].copy()
desert_albedo.rename({'albedo': 'albedo_soil'}, axis=1, inplace=True)
display(desert_albedo.loc[desert_albe])

# Merge desert soil albedo
ketura = ketura.merge(desert_albedo, how='left',  on=['Season','time'])

# Calculate shade albedo
ketura['albedo_shade'] = ketura['albedo_soil'] * ketura['f_dif']

# Calculate PV field albedo, use the different albedos
ketura.loc[ketura['Ecosystem'] == 'PV field', 'albedo'] = ketura.loc[ketura['Ecosystem'] == 'PV field', 'albedo_soil'] \
                                                        * ketura.loc[ketura['Ecosystem'] == 'PV field', 'soil'] \
                                                        + ketura.loc[ketura['Ecosystem'] == 'PV field', 'albedo_shade'] \
                                                        * ketura.loc[ketura['Ecosystem'] == 'PV field', 'shadow'] \
                                                        + albedo_pv \
                                                        * ketura.loc[ketura['Ecosystem'] == 'PV field', 'panel']

# Convert albedo to fraction, not %
ketura['albedo'] = ketura['albedo']/100

#display(ketura.loc[~ketura['f_dif'].isna()])

#print(ketura.columns.values)

,time,Season,albedo_soil
0,00:45,Summer,NaN
1,01:15,Summer,NaN
2,01:45,Summer,NaN
3,02:15,Summer,NaN
4,02:45,Summer,NaN
...,...,...,...
748,21:45,Autumn,NaN
749,22:15,Autumn,NaN
750,22:45,Autumn,NaN
751,23:15,Autumn,NaN


In [7]:
def weighted_avg_and_std(values, weights):
    values = np.ma.masked_invalid(values)
    # Return the weighted average and standard deviation.
    average = np.ma.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.ma.average((values-average)**2, weights=weights)
    
    maximum = np.max(values)
    minimum = np.min(values)
    return (average, np.sqrt(variance), maximum, minimum)

print('Summer:')
summer = ketura.loc[(ketura['Ecosystem'] == 'PV field') & (ketura['Season'] == 'Summer')]
print(weighted_avg_and_std(summer['albedo'], summer['SW_IN_merge']))
print()

print('Autumn:')
autumn = ketura.loc[(ketura['Ecosystem'] == 'PV field') & (ketura['Season'] == 'Autumn')]
print(weighted_avg_and_std(autumn['albedo'], autumn['SW_IN_merge']))

Summer:
(0.19880982835997008, 0.003966225302099919, 0.20793598124994594, 0.19278360029596173)

Autumn:
(0.14105746107276582, 0.027984517948550423, 0.23761007637864348, 0.0950338235120522)


## Trigonometric calculations with position of the sun

In [41]:
# Assume trigonometric calculations based on position of the sun, i.e. shade does not count:
# Normalise area based on "no shade" scenario:

# "Flatten" the pv panels
panel_angle = 35
perc_df['panel_flat'] = perc_df['panel'] / np.cos(np.radians(panel_angle))

# Normalise the % to 100%, because sometimes it only reaches 96% total
perc_df['total'] = perc_df['panel_flat'] + perc_df['soil']
perc_df['panel_norm']  = perc_df['panel_flat'] * 100 / perc_df['total']
perc_df['soil_norm']   = perc_df['soil'] * 100 / perc_df['total']
perc_df.drop(['total'], axis=1, inplace=True)

#display(perc_df)

In [37]:
# Collapse percentages to diurnals
perc_df['DateTime'] = pd.to_datetime(perc_df['time'], format='%H:%M')

# set the DateTime column as the index
perc_df.set_index('DateTime', inplace=True)
# resample the data to 15-minute intervals and apply a smoothing function (e.g., rolling mean)
df_resampled = perc_df.groupby(['Ecosystem', 'Season']).resample('15T').mean().rolling(window=6, min_periods=3).mean()
df_resampled.reset_index(inplace=True)

df_resampled['time'] = df_resampled['DateTime'].dt.strftime('%H:%M')
col = df_resampled.pop('time')
df_resampled.insert(0, col.name, col, allow_duplicates=True)
df_resampled.drop(['DateTime'], axis=1, inplace=True)

perc_df.reset_index(inplace=True)
#display(df_resampled)

In [38]:
# Calculate diffuse fraction in Yatir (Assuming it's the same in Ketura)
yatir_df['f_dif'] = yatir_df['S_top_diff(CM21_V)_Wm-2'] / yatir_df['S_top_atm(CM21_IV)_Wm-2']

# Add diffuse fraction from Yatir to Ketura
ketura = ketura_df.merge(yatir_df[['DateTime','f_dif']], how='left', on=['DateTime'])

# Add percentages
ketura = ketura.merge(df_resampled, how='left',  on=['Ecosystem','Season','time'])

# Fix bad SW values
ketura.loc[ketura['SW_IN_merge'] < 0, 'SW_IN_merge'] = 0
ketura.loc[ketura['SW_OUT_merge'] < 0, 'SW_OUT_merge'] = 0
ketura.loc[ketura['SW_IN_merge'] <= 0, 'SW_OUT_merge'] = 0

# Re-calculate albedo from SW measurements (relevant for the desert)
ketura['albedo'] = np.nan
ketura['albedo'] = ketura['SW_OUT_merge'] / ketura['SW_IN_merge']

# Extract desert albedo & rename. This will be used for the soil & shade fractions of the PV field
desert_albedo = ketura.loc[ketura['Ecosystem'] == 'Desert background',['time','Season','albedo']].copy()
desert_albedo.rename({'albedo': 'albedo_soil'}, axis=1, inplace=True)

# Merge desert soil albedo
ketura = ketura.merge(desert_albedo, how='left',  on=['Season','time'])

# Calculate shade albedo
ketura['albedo_shade'] = ketura['albedo_soil'] * ketura['f_dif']

# Calculate PV field albedo, use the different albedos
ketura.loc[ketura['Ecosystem'] == 'PV field', 'albedo2'] = ketura.loc[ketura['Ecosystem'] == 'PV field', 'albedo_soil'] \
                                                        * ketura.loc[ketura['Ecosystem'] == 'PV field', 'soil_norm'] \
                                                        + albedo_pv \
                                                        * ketura.loc[ketura['Ecosystem'] == 'PV field', 'panel_norm']

# Convert albedo to fraction, not %
ketura['albedo2'] = ketura['albedo2']/100

#display(ketura.loc[~ketura['f_dif'].isna()])

#print(ketura.columns.values)

In [40]:
def weighted_avg_and_std(values, weights):
    values = np.ma.masked_invalid(values)
    # Return the weighted average and standard deviation.
    average = np.ma.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.ma.average((values-average)**2, weights=weights)
    
    maximum = np.max(values)
    minimum = np.min(values)
    return (average, np.sqrt(variance), maximum, minimum)

print('Summer:')
summer = ketura.loc[(ketura['Ecosystem'] == 'PV field') & (ketura['Season'] == 'Summer')]
print(weighted_avg_and_std(summer['albedo2'], summer['SW_IN_merge']))
print()

print('Autumn:')
autumn = ketura.loc[(ketura['Ecosystem'] == 'PV field') & (ketura['Season'] == 'Autumn')]
print(weighted_avg_and_std(autumn['albedo2'], autumn['SW_IN_merge']))

Summer:
(0.18555025670686953, 0.002397460075537444, 0.19121491012888597, 0.18151490484608554)

Autumn:
(0.15112860528884273, 0.013519825084567942, 0.19043474960446594, 0.12644931128617004)
